# Bandgap Workflow Example
 This notebook demonstrates how to build and run a bandgap workflow for a material.

## Process Overview
### 1. Set up the environment and parameters.
### 1. Log in to get the API token
### 1. Load the target material.
### 1. Import workflow builder and related analyzers.
### 1. Analyze material to get parameters for the workflow configuration.
### 1. Create the workflow configuration.
### 1. Create a job with material and workflow configuration.
### 1. Submit the job to the server.
### 1. Monitor the job status and retrieve results.

## 1. Set up the environment and parameters

## 2. Log in to get the API token

In [2]:
from mat3ra.api import ApiClient
# Log in to get the API token
auth_config = await ApiClient().login()

ModuleNotFoundError: No module named 'mat3ra.api'

## 3. Load the target material

In [ ]:
from utils.visualize import visualize_materials as visualize
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder("/uploads", "MoS2_twisted_interface_60_degrees.json")
visualize(material)

## 4. Import workflow builder and related analyzers

In [ ]:
from mat3ra.wode.analyzers.electronic import KPointAnalyzer, CutoffAnalyzer, SmearingAnalyzer, BandsAnalyzer

kpoint_analyzer = KPointAnalyzer(material=material)
cutoff_analyzer = CutoffAnalyzer(material=material)
smearing_analyzer = SmearingAnalyzer(material=material)
bands_analyzer = BandsAnalyzer(material=material)

kpoints = kpoint_analyzer.get_kpoints()
cutoff = cutoff_analyzer.get_cutoff()
smearing = smearing_analyzer.get_smearing()
number_of_bands = bands_analyzer.get_number_of_bands()



## 5. Get workflow and adjust its parameters

In [ ]:
from mat3ra.wode.workflow.vasp.band_structure import BandStructureWorkflow
from mat3ra.wode.pseudopotentials import PseudopotentialEnum

workflow = BandStructureWorkflow()
workflow.set_kpoints(kpoints)
workflow.set_pseudopotential(PseudopotentialEnum.PAW_HSE) # elements will be set automatically based on the material

## 6. Create the job configuration

In [ ]:
from mat3ra.wode.compute import ComputeConfiguration, QueueEnum
compute_config = ComputeConfiguration(
    queue = QueueEnum.OR8,
    nodes = 1,
    cores = 8,
)

## 7. Submit the job and monitor the status

In [ ]:
from mat3ra.wode.job import create_job
job = create_job(workflow=workflow, material=material, compute = compute_config, auth_config=auth_config)
job.run()
job.wait_for_complete()
# job.check_status()
# job.get_current_output()
# AFTER Finished
job.get_results(PropertyEnum.BANDGAP)